# Создание единого слоя нейронной сети с использованием `Flux.jl`

В этом блокноте мы выйдем за рамки двоичной классификации. Сейчас мы попытаемся различить три плода, а не два. Мы сделаем это, используя **несколько** нейронов, расположенных в **одном слое**.

## Считывание и обработка данных

Мы можем начать с загрузки необходимых пакетов и приведения наших данных в рабочее состояние с помощью аналогичного кода, который мы использовали в начале предыдущих записных книжек, за исключением того, что теперь мы объединим три разных набора данных яблок и добавим немного винограда в фруктовый салат!

In [ ]:
using CSV, DataFrames, DelimitedFiles

In [ ]:
# Load apple data in with `readdlm` for each file
apples1, applecolnames1 = readdlm("data/Apple_Golden_1.dat", '\t', header = true)
apples2, applecolnames2 = readdlm("data/Apple_Golden_2.dat", '\t', header = true)
apples3, applecolnames3 = readdlm("data/Apple_Golden_3.dat", '\t', header = true)

# Check that the column names are the same for each apple file
println( applecolnames1 == applecolnames2 == applecolnames3)

Поскольку каждый файл apple имеет столбцы с одинаковыми заголовками, мы знаем, что можем объединить эти столбцы из разных файлов вместе:

In [ ]:
apples = vcat(apples1, apples2, apples3)

А теперь давайте создадим массив с именем `x_apples`, в котором будут храниться данные из столбцов` apple` и `red` и` blue`. Из `applecolnames1` мы можем видеть, что это 3-й и 5-й столбцы` apple`:

In [ ]:
applecolnames1

In [ ]:
length(apples[:, 1])

In [ ]:
x_apples  = [ [apples[i, 3], apples[i, 5]] for i in 1:length(apples[:, 3]) ]

А теперь давайте создадим массив с именем `x_apples`, в котором будут храниться данные из столбцов` apple` и `red` и` blue`. Из `applecolnames1` мы можем видеть, что это 3-й и 5-й столбцы` apple`:

In [ ]:
# Загрузить данные из *.dat файлов
bananas, bananacolnames = readdlm("data/Banana.dat", '\t', header = true)
grapes1, grapecolnames1 = readdlm("data/Grape_White.dat", '\t', header = true)
grapes2, grapecolnames2 = readdlm("data/Grape_White_2.dat", '\t', header = true)

# Объединить данные из двух файлов винограда вместе
grapes = vcat(grapes1, grapes2)

# Убедитесь, что столбец 3 и столбец 5 ссылаются на «красный» и «синий» столбцы из каждого файла
println("All column headers are the same: ", bananacolnames == grapecolnames1 == grapecolnames2 == applecolnames1)

# Сборка x_bananas и x_grapes из бананов и винограда
x_bananas  = [ [bananas[i, 3], bananas[i, 5]] for i in 1:length(bananas[:, 3]) ]
x_grapes = [ [grapes[i, 3], grapes[i, 5]] for i in 1:length(grapes[:, 3]) ]

## One-hot вектора

Теперь мы хотим классифицировать *три* разных вида фруктов. Не ясно, как кодировать эти три типа, используя одну выходную переменную; действительно, вообще это невозможно. (?можно воспринимать 0-0.2->яблоко 0.45-0.65->банан 0.8-1.0->виноград...)

Вместо этого у нас есть идея кодирования выходных типов $ n $ из классификации в * векторы длины $ n $ *, называемые «горячими векторами»:

$$
\textrm{apple} = \begin{pmatrix} 1 \\ 0 \\ 0 \end{pmatrix};
\quad
\textrm{banana} = \begin{pmatrix} 0 \\ 1 \\ 0 \end{pmatrix};
\quad
\textrm{grape} = \begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix}.
$$

Термин "one-hot" относится к тому факту, что каждый вектор имеет $1$ или $0$.

По сути, первый нейрон узнает, соответствует ли (1 или 0) данным яблоко, второй - соответствует ли (1 или 0) банану и т.д.

Flux.jl имеет эффективное представление для горячих векторов, используя расширенные возможности Джулии, так что он фактически не хранит эти векторы, что будет пустой тратой памяти; вместо этого `Flux` просто записывает, в каком положении находится ненулевой элемент. Для нас, однако, похоже, что вся информация хранится в виде:

In [ ]:
using Flux: onehot

onehot(1, 1:3)

#### Упражнение 1

Создайте массив меток, который дает метки (1, 2 или 3) каждой точки данных. Затем используйте `onehot`, чтобы закодировать информацию о метках как вектора` OneHotVector`.

## Один слой в Flux

Предположим, что есть две части входных данных, как в предыдущем блокноте с одним нейроном. Тогда сеть имеет 2 входа и 3 выхода:

In [ ]:
include("draw_neural_net.jl")
draw_network([2, 3])

`Flux` позволяет нам выразить это снова простым способом:

In [ ]:
using Flux

In [ ]:
model = Dense(2, 3, σ)

#### Упражнение 2

Теперь, как выглядят веса внутри `модели`? Как это соотносится с диаграммой сетевого уровня выше?

## Обучение модели

Несмотря на то, что модель теперь сложнее, чем одиночный нейрон из предыдущего ноутбука, прелесть Flux.jl в том, что остальная часть тренировочного процесса **выглядит точно так же**!

#### Упражнение 3

Реализуйте обучения для этой модели.

#### Упражнение 4

Визуализируйте результат обучения для каждого нейрона. Поскольку каждый нейрон является сигмоидальным, мы можем получить хорошее представление о функции, просто нанося на график один контурный уровень, где функция принимает значение 0,5, используя функцию «контур» с ключевым аргументом `level = [0.5, 0.501]`.

In [ ]:
using Plots
plotly()

In [ ]:
plot()

contour!(0:0.01:1, 0:0.01:1, (x,y)->model([x,y]).data[1], levels=[0.5, 0.501], color = cgrad([:blue, :blue]))
contour!(0:0.01:1, 0:0.01:1, (x,y)->model([x,y]).data[2], levels=[0.5,0.501], color = cgrad([:green, :green]))
contour!(0:0.01:1, 0:0.01:1, (x,y)->model([x,y]).data[3], levels=[0.5,0.501], color = cgrad([:red, :red]))

scatter!(first.(x_apples), last.(x_apples), m=:cross, label="apples")
scatter!(first.(x_bananas), last.(x_bananas), m=:circle, label="bananas")
scatter!(first.(x_grapes), last.(x_grapes), m=:square, label="grapes")

#### Упражнение 5

Интерпретируйте результаты, проверяя, какие плоды должен был выучить каждый нейрон и чего ему удалось достичь.